In [1]:
import os
import pandas as pd
from kernels.wrapper import MODELS, KernelModelWrapper
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, get_test_ipc2023_learning_instance_files, get_best_bound
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from util.visualise import *

In [2]:
pd.options.display.float_format = "{:.1f}".format

In [3]:
_LOG_DIR = "logs/test_ipc2023"
_DIFFICULTIES = ["easy", "medium", "hard"]
_ITERATIONS = [str(i) for i in [1]]
_MODELS = ["linear-regression", "linear-svr", "ridge", "lasso", "rbf-svr", "quadratic-svr", "cubic-svr"]
CONFIGS = list(product(_MODELS, _ITERATIONS))

In [4]:
def get_data(domain):
  data = {
  }

  test_files = get_test_ipc2023_learning_instance_files()

  for config in CONFIGS:
    d = {
      "solved": [],
      "expanded": [],
      "cost": [],
      "first_h": [],
      "quality": [],
      "c_opt": [],
      "difficulty": [],
    }
    model, iterations = config

    for difficulty in _DIFFICULTIES:
      for df, pf in test_files[domain]:
        if not difficulty in pf:  # TODO generalise
          continue
        pf_name = os.path.basename(pf).replace(".pddl", "")
        log_file = f"{_LOG_DIR}/{domain}_{difficulty}_{pf_name}_trained_models_kernel-{model}_llg_ipc2023-learning-{domain}_wl_{iterations}_0.joblib.log"
        
        if not os.path.exists(log_file):
          continue

        stats = scrape_search_log(log_file)
        for k in d:
          if k in stats:
            d[k].append(stats[k])
        
        c = stats["cost"]
        c_opt = get_best_bound(domain, difficulty, pf_name)
        quality = c_opt / c
        quality = max(quality, 0)
        d["quality"].append(quality)
        d["c_opt"].append(c_opt)
        d["difficulty"].append(difficulty)
        
    data[config] = d

  return data

In [5]:
def get_metrics_df(data):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  for config in CONFIGS:
    d["config"].append("_".join(config))
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in _DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += 1
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [6]:
def get_table_df():
  d = {
    "domain": [],
  }
  for config in CONFIGS:
    d['_'.join(config)] = []

  sum_scores = {config: 0 for config in CONFIGS}

  for domain in IPC2023_LEARNING_DOMAINS:
    data = get_data(domain)
    d["domain"].append(domain)
    for config in CONFIGS:
      score = sum(data[config]["quality"])
      d['_'.join(config)].append(score)
      sum_scores[config] += score
  

  d["domain"].append("sum")
  for config in CONFIGS:
    d['_'.join(config)].append(sum_scores[config])

  return pd.DataFrame(d)

In [7]:
display(get_table_df())

,domain,linear-regression_1,linear-svr_1,ridge_1,lasso_1,rbf-svr_1,quadratic-svr_1,cubic-svr_1
0,blocksworld,0,14.7,15.8,6.6,4.7,8.6,8.8
1,childsnack,0,6.0,6.8,6.0,4.3,3.8,4.6
2,ferry,0,8.7,9.0,9.0,9.9,30.5,30.5
3,floortile,0,1.8,1.8,0.0,0.0,0.9,0.9
4,miconic,0,29.1,29.0,30.0,29.4,28.9,29.1
5,rovers,0,10.1,10.2,10.5,10.0,9.4,8.3
6,satellite,0,13.3,12.5,11.3,4.6,9.6,9.6
7,sokoban,0,15.8,15.8,22.0,21.0,21.0,21.0
8,spanner,0,30.0,30.0,30.0,30.0,30.0,30.0
9,transport,0,25.6,9.0,8.0,7.9,14.9,25.1


In [8]:
for domain in IPC2023_LEARNING_DOMAINS:
  print(domain)
  data = get_data(domain)
  display(get_metrics_df(data))

blocksworld


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,0.0,0,0,0.0,0,0,0,0,0,0,0,0
1,linear-svr_1,14.7,27,30,14.7,27,30,0,0,0,0,0,0
2,ridge_1,15.8,28,30,15.8,28,30,0,0,0,0,0,0
3,lasso_1,6.6,7,30,6.6,7,30,0,0,0,0,0,0
4,rbf-svr_1,4.7,6,30,4.7,6,30,0,0,0,0,0,0
5,quadratic-svr_1,8.6,16,30,8.6,16,30,0,0,0,0,0,0
6,cubic-svr_1,8.8,18,30,8.8,18,30,0,0,0,0,0,0


childsnack


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,0.0,0,0,0.0,0,0,0,0,0,0,0,0
1,linear-svr_1,6.0,8,30,6.0,8,30,0,0,0,0,0,0
2,ridge_1,6.8,8,30,6.8,8,30,0,0,0,0,0,0
3,lasso_1,6.0,8,30,6.0,8,30,0,0,0,0,0,0
4,rbf-svr_1,4.3,5,30,4.3,5,30,0,0,0,0,0,0
5,quadratic-svr_1,3.8,5,30,3.8,5,30,0,0,0,0,0,0
6,cubic-svr_1,4.6,5,30,4.6,5,30,0,0,0,0,0,0


ferry


...

,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,0.0,0,0,0.0,0,0,0,0,0,0,0,0
1,linear-svr_1,15.8,17,30,15.8,17,30,0,0,0,0,0,0
2,ridge_1,15.8,17,30,15.8,17,30,0,0,0,0,0,0
3,lasso_1,22.0,22,30,22.0,22,30,0,0,0,0,0,0
4,rbf-svr_1,21.0,21,30,21.0,21,30,0,0,0,0,0,0
5,quadratic-svr_1,21.0,21,30,21.0,21,30,0,0,0,0,0,0
6,cubic-svr_1,21.0,21,30,21.0,21,30,0,0,0,0,0,0


spanner


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,0.0,0,0,0.0,0,0,0,0,0,0,0,0
1,linear-svr_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0
2,ridge_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0
3,lasso_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0
4,rbf-svr_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0
5,quadratic-svr_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0
6,cubic-svr_1,30.0,30,30,30.0,30,30,0,0,0,0,0,0


transport


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,linear-regression_1,0.0,0,0,0.0,0,0,0,0,0,0,0,0
1,linear-svr_1,25.6,23,30,25.6,23,30,0,0,0,0,0,0
2,ridge_1,9.0,9,30,9.0,9,30,0,0,0,0,0,0
3,lasso_1,8.0,8,30,8.0,8,30,0,0,0,0,0,0
4,rbf-svr_1,7.9,8,30,7.9,8,30,0,0,0,0,0,0
5,quadratic-svr_1,14.9,14,30,14.9,14,30,0,0,0,0,0,0
6,cubic-svr_1,25.1,23,30,25.1,23,30,0,0,0,0,0,0


In [49]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    for config in CONFIGS:
      cs = np.array(data[config]["cost"])
      c_opts = np.array(data[config]["c_opt"])
      index = np.where(cs != -1)
      cs = cs[index]
      c_opts = c_opts[index]
      plt.scatter(c_opts, cs, label="_".join(config))
      max_val = max([max_val, max(cs), max(c_opts)])
    
    plt.title(domain)
    plt.plot([1, max_val], [1, max_val], alpha=0.5, linestyle="dotted", c='k')
    plt.ylabel("C")
    plt.xlabel("C* (not necessarily optimal)")
    plt.legend()
    # plt.show()
    plt.savefig(f"plots/ipc2023-learning-{domain}-cost.png")
  except:
    pass
  plt.clf()
print("c vs c* saved")

c vs c* saved


<Figure size 640x480 with 0 Axes>

In [50]:
os.makedirs("plots", exist_ok=True)
for domain in IPC2023_LEARNING_DOMAINS:
  try:
    data = get_data(domain)

    max_val = 0

    my_dict = {}

    fig, ax = plt.subplots()

    for config in CONFIGS:
      expanded = np.array(data[config]["expanded"])
      # index = np.where(expanded != -1)
      expanded = expanded[index]
      my_dict["_".join(config)] = expanded
    
    plt.yscale("log")
    plt.title(domain)
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
    # plt.ylim(bottom=1)
    # plt.show()
    plt.savefig(f"plots/ipc2023-learning-{domain}-expanded.png")
  except:
    pass
  plt.clf()
print("expansions saved")

expansions saved


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>